In [1]:
import gzip
import shutil
import os
import pickle
import pandas as pd
from collections import defaultdict
import numpy as np

In [2]:
download_folder = f"D:\\Research\\Test_data"
all_data = []

# Hàm tạo ma trận kề

In [3]:
# https://github.com/liyaguang/DCRNN/blob/master/scripts/gen_adj_mx.py

def get_adjacency_matrix(distance_df, sensor_ids, normalized_k=0.1):
    """

    :param distance_df: data frame with three columns: [from, to, distance].
    :param sensor_ids: list of sensor ids.
    :param normalized_k: entries that become lower than normalized_k after normalization are set to zero for sparsity.
    :return:
    """
    num_sensors = len(sensor_ids)
    dist_mx = np.zeros((num_sensors, num_sensors), dtype=np.float32)
    dist_mx[:] = np.inf
    # Builds sensor id to index map.
    sensor_id_to_ind = {}
    for i, sensor_id in enumerate(sensor_ids):
        sensor_id_to_ind[sensor_id] = i

    # Fills cells in the matrix with distances.
    for row in distance_df.values:
        if row[0] not in sensor_id_to_ind or row[1] not in sensor_id_to_ind:
            continue
        dist_mx[sensor_id_to_ind[row[0]], sensor_id_to_ind[row[1]]] = row[2]

    # Calculates the standard deviation as theta.
    distances = dist_mx[~np.isinf(dist_mx)].flatten()
    std = distances.std()
    adj_mx = np.exp(-np.square(dist_mx / std))
    # Make the adjacent matrix symmetric by taking the max.
    # adj_mx = np.maximum.reduce([adj_mx, adj_mx.T])

    adj_mx_test = np.exp(-np.square(dist_mx / std))
    print(np.sum(adj_mx_test >= normalized_k) / adj_mx_test.size)

    # Sets entries that lower than a threshold, i.e., k, to zero for sparsity.
    adj_mx[adj_mx < normalized_k] = 0
    return sensor_ids, sensor_id_to_ind, adj_mx


# 1. Hàm đọc dữ liệu từ các file zip

## 1.1 Đọc và xử lí dữ liệu từ file zip

In [4]:
# for filename in os.listdir(download_folder):
#     if filename.endswith(".txt.gz"):
#         gz_path = os.path.join(download_folder, filename)
        
#         with gzip.open(gz_path, 'rb') as f_in:
#             with open('temp.txt', 'wb') as f_out:
#                 shutil.copyfileobj(f_in, f_out)
        
#         data = pd.read_table("temp.txt", sep=",", header=None)
#         data = data.iloc[:, :12]
#         data.columns = ["timestamp", "station", "district", "freeway", "direction", "lanetype",
#         "station_length", "samples", "observed_percentage", "total_flow", "avg_occ", "avg_speed"]
#         data = data[data['lanetype'] == 'ML']
#         data = data[["timestamp", "station", "observed_percentage", "total_flow", "avg_occ", "avg_speed"]]
#         data["timestamp"] = pd.to_datetime(data["timestamp"])
#         all_data.append(data)
#         print(filename)

# with open("all_data.pkl", "wb") as f:
#     pickle.dump(all_data, f)

## 1.2 Đọc dữ liệu từ file pkl(dữ liệu trong các file zip đã được đọc và lưu lại)

In [5]:
with open("all_data.pkl", "rb") as f:
    all_data = pickle.load(f)

# 2. Xử lí dữ liệu

## 2.1 Lấy các mẫu được quan sát hơn 1 tỉ lệ cho trước và xuất hiện liên tục 1 khoảng thời gian

In [6]:
# Lọc lấy các mẫu có tỉ lệ ghi nhận cao(đáng tin cậy), loại bỏ các mẫu nội suy nhiều
data = [df[df['observed_percentage'] >= 99] for df in all_data]

# Lọc lấy danh sách các sensor xuất hiện nhiều(liên tục 1 khoảng thời gian)
min_consecutive_days = 60
sensor_streaks = defaultdict(int)
max_streaks = defaultdict(int)

for df in data:
    sensors_today = set(df['station'].unique())
    
    for sensor in sensors_today:
        sensor_streaks[sensor] += 1
        max_streaks[sensor] = max(max_streaks[sensor], sensor_streaks[sensor])
    
    missing_sensors = set(sensor_streaks.keys()) - sensors_today
    for sensor in missing_sensors:
        sensor_streaks[sensor] = 0

valid_sensors = {s for s, streak in max_streaks.items() if streak >= min_consecutive_days}
print(f"Số sensor giữ lại: {len(valid_sensors)}")

Số sensor giữ lại: 1062


## 2.2 Đọc metadata các sensor, xóa các sensor trùng vị trí địa lí và các sensor đã loại ở bước trên. Chỉ lấy phần dữ liệu còn lại

In [ ]:
# Đọc metadata lấy danh sách các sensor, lấy các sensor của đường "ML", lấy các sensor trong danh sách valid_sensor, loại bỏ các sensor trùng địa điểm(cùng kinh độ và vĩ độ)
meta = pd.read_csv("../Meta_District4.txt", sep='\t')
meta = meta[meta['Type'] == 'ML']
meta = meta[meta['ID'].isin(valid_sensors)]
meta = (
    meta
    .sort_values('ID')  # Đảm bảo station nhỏ nhất lên trước
    .drop_duplicates(subset=['Latitude', 'Longitude'], keep='first')
    .reset_index(drop=True)
)
# Lấy danh sách valid_sensor mới
sensors_df = meta[['ID', 'Latitude', 'Longitude']]
# Lấy các dữ liệu từ các sensor hợp lệ
raw_data = [df[df['station'].isin(valid_sensors)] for df in data]
raw_data = pd.concat(raw_data, ignore_index=True)
raw_data = raw_data.sort_values(['timestamp', 'station'])
raw_data['timestamp'] = pd.to_datetime(raw_data['timestamp'])

# raw_data.to_csv('Raw_Data.csv', index = False)
# np.save("sensors_df.npy", np.array(sensors_df))

## 2.3 Đọc dữ liệu đã xử lí

In [9]:
# raw_data = pd.read_csv('Raw_Data.csv')
# raw_data['timestamp'] = pd.to_datetime(raw_data['timestamp'])
# sensors_df = np.load("sensors_df.npy")

# 3. Chuyển thành numpy array

## 3.1 Xử lí DF, tạo mốc thời gian, tạo mảng numpy array. Gán các phần tử trong DF vào array

In [ ]:
# Tạo các mốc thời gian
start_time = raw_data["timestamp"].min()
end_time = raw_data["timestamp"].max()
full_time_index = pd.date_range(start=start_time, end=end_time, freq="5min")

In [11]:
time_to_ind = {t: i for i, t in enumerate(full_time_index)}
sensor_id_to_ind = {s: i for i, s in enumerate(sensors_df["ID"].values)}

In [12]:
result = np.full((len(time_to_ind), len(sensor_id_to_ind), 3), np.nan)

In [13]:
for row in raw_data.itertuples(index=False):
    x = time_to_ind.get(row.timestamp) 
    y = sensor_id_to_ind.get(row.station)
    result[x, y, 0] = row.total_flow
    result[x, y, 1] = row.avg_speed
    result[x, y, 2] = row.avg_occ

In [14]:
print(np.isnan(result).sum())
print(result.shape)
total_elements = result.size
total_nan = np.isnan(result).sum()
nan_ratio = total_nan / total_elements * 100
print(total_elements)
print(nan_ratio)

4062192
(34848, 1053, 3)
110084832
3.6900560469584036


In [ ]:
np.save("/PEMSBAY.npy", result)

## 3.2 Đọc dữ liệu numpy array

In [ ]:
# data_array = np.load("/PEMSBAY.npy")

# 4. Tạo adj_mx

In [18]:
distance = []
for i in range(len(sensors_df) - 1):
    for j in range(i + 1, len(sensors_df)):
        dis = ((sensors_df["Latitude"][i] - sensors_df['Latitude'][j]) ** 2 + (sensors_df['Longitude'][i] - sensors_df['Longitude'][j]) ** 2) ** 0.5
        distance.append([sensors_df['ID'][i], sensors_df['ID'][j], dis])

In [19]:
distance_df = pd.DataFrame(distance, columns=['from', 'to', 'distance'])
sensors_ids = sensors_df['ID']

In [20]:
sensor_ids, sensor_id_to_ind, adj_mx = get_adjacency_matrix(distance_df, sensors_ids, 0.1)

0.27477230072988224


In [ ]:
to_save = (sensor_ids, sensor_id_to_ind, adj_mx)
with open("/adj_mx_bay.pkl", "wb") as f:
    pickle.dump(to_save, f)